In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import strawberryfields as sf
from strawberryfields.ops import *
import matplotlib
import pennylane as qml
import flatnetwork_simple as fns

# Working Circuit Embedding

In [27]:
#Setup
K = 20
dt = 1.0

#Parameters
mu = lambda t,i: 1.0
rho = 1.0

#Annealing params
J = 1.0
theta = -J*dt/K

L = [3,1,1]
d = 3
fnet = fns.FlatNetwork(L,d)
fnet.loadParams(mu, rho)
fnet.make_mpos()

S, D = fnet.getHamiltonian()

features = [1., 2.]
layer1   = [S[i] for i in S]
layer2   = [D[i] for i in D]
weights  = [layer1, layer2]

import pennylane as qml
from pennylane.templates import QAOAEmbedding

dev = qml.device('default.qubit', wires=2)

@qml.qnode(dev)
def circuit(weights, f=None):
    QAOAEmbedding(features=f, weights=weights, wires=range(2))
    return qml.expval(qml.PauliZ(0))

print(circuit(weights, f=features))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
